In [34]:
import csv
import pandas
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import matplotlib
import seaborn as sns
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

users = pandas.read_csv("data/users.csv", header=None)
conversions = pandas.read_csv("data/conversions.csv", header=None)
items = pandas.read_csv("data/items.csv", header=None)
users_ads = pandas.read_csv("data/users_ads.csv", header=None)
# views = pandas.read_csv("data/views.csv", header=None)

Renaming dataframes columns for comfy reading

In [35]:
users.columns = ['userId', 'registerCountry', 'signupTime']
conversions.columns = ['userId', 'itemId', 'price', 'quantity', 'timestamp']
items.columns = ['itemId', 'style', 'personality', 'color', 'theme', 'price', 'category']
users_ads.columns = ['userId', 'utmSource', 'utmCampaign', 'utmMedium', 'utmTerm', 'utmContent']
# views.columns = ['userId', 'itemId', 'timestamp', 'pagetype']

In [3]:
users.signupTime = pandas.to_datetime(users.signupTime)
conversions.timestamp = pandas.to_datetime(conversions.timestamp)
# views.timestamp = pandas.to_datetime(views.timestamp)

I only am doing it to make the work on the charts a bit faster: choosing only 20% of each dataset.

In [42]:
import random
def random_subset(df, percent):
    new_size  = int(0.2*len(df.index))
    subset = random.sample(set(df.index), new_size)
    return df.ix[subset]

users_small = random_subset(users, 0.2)
items_small = random_subset(items, 0.2)
users_ads_small = random_subset(users_ads, 0.2)
conversions_small = random_subset(conversions, 0.2)
# views_small = random_subset(views, 0.2)

A simple plot showing how many people (participating in any conversion) bought particular number of items.

In [44]:
# for each user, how many items he purchased
counts = conversions.userId.value_counts()
# for each count of purchases/client, how many is them?
# so: how many clients purchased k items
counts2 = counts.value_counts()
counts2 = counts2.sort_index()
# tmp = pandas.Series(counts.index, index=counts.values)
counts2.plot()
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 1668 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


Plot saying when users registered.

In [ ]:
df_per_month = users.signupTime.value_counts()
df_per_month = df_per_month.sort_index()
df_per_month.plot(x=range(1,13))
plt.show()

Evolve timestamp to year, month, day and hour.

In [8]:
def add_year_to_hr(df, timename):
    df['month'] = pandas.DatetimeIndex(df[timename]).month
    df['year'] = pandas.DatetimeIndex(df[timename]).year
    df['day'] = pandas.DatetimeIndex(df[timename]).day
    df['hour'] = pandas.DatetimeIndex(df[timename]).hour
    return df

How much ppl spend each hour?

In [17]:
conversions = add_year_to_hr(conversions, "timestamp")
users = add_year_to_hr(users, "signupTime")

In [13]:
sum_per_hour = []
for i in range(0,24):
    sum_per_hour.append(conversions[conversions.hour == i].price.sum())

pandas.Series(sum_per_hour).plot()
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 759 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


and each day of the month

In [ ]:
sum_per_day = []
for i in range(1,32):
    sum_per_day.append(conversions[conversions.day == i].price.sum())

pandas.Series(sum_per_day).plot()
plt.show()

How many users registered each month?

In [18]:
users_per_month = users.month.value_counts()
users_per_month = users_per_month.sort_index()
users_per_month.plot(x=range(1,13))
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 1157 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


What time do users usually register?

In [25]:
sum_per_hour = []
for i in range(0,24):
    sum_per_hour.append(users[users.hour == i].hour.size)

pandas.Series(sum_per_hour).plot()
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 1324 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


Fist attempt to perform PCA (analysis of items price, cathegory and color split by theme). It is not working correctly yet, did not manage to fix it.

In [43]:
X = np.array([items_small.price.values,
              items_small.category.values,
              items_small.color.values])

y = np.array([items_small.theme.values])

fig = plt.figure(1)
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()
pca = decomposition.PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

# y = np.choose(y, [1, 2]).astype(np.float)
ax.scatter(X[0], X[1], X[2], c=[1,2], cmap=plt.cm.spectral)

x_surf = [X[:, 0].min(), X[:, 0].max(),
          X[:, 0].min(), X[:, 0].max()]
y_surf = [X[:, 0].max(), X[:, 0].max(),
          X[:, 0].min(), X[:, 0].min()]
x_surf = np.array(x_surf)
y_surf = np.array(y_surf)
v0 = pca.transform(pca.components_[[0]])
v0 /= v0[-1]
v1 = pca.transform(pca.components_[[1]])
v1 /= v1[-1]

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 1608 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


Merging dataframes - creating a frame containing full information about transactions (with redundancies, so cannot be applied to views, as it is too big). Will be useful for a single customer journey visualization I am working on now.

In [ ]:
full_info_conversions = conversions.merge(users, how='inner', on='userId')
full_info_conversions = full_info_conversions.merge(items, how='inner', on='itemId')